In [1]:
from math import log2

def sgbp_lower_bound(n, k):
    if 2**k >= n:
        return 2**k + n
    else:
        return 2**k * 2**(n/(2**k))
        
def lgbp_lower_bound(n = 288, k = 2, pow2 = True):
    if pow2:
        K = 2**k
    else:
        K = k
    if K >= 2**20:
        st = K + n//log(K,2)
        while binomial(st, st - K) < 2**n:
            st += 1
        return st
    st = n//K
    while binomial(2**st, K) < 2**n:
        st += 1
    if binomial(2**st, K) == 2**n:
        return 2**st
    # binary search
    ub = 2**(st)
    lb = 2**(st - 1)
    while lb + 1 < ub:
        mid = (ub + lb) // 2
        total = binomial(mid, K)
        if total < 2**n:
            lb = mid
        elif total > 2**n:
            ub = mid
        else:
            return mid
    assert binomial(lb, K) < 2**n
    assert binomial(ub, K) >= 2**n
    return ub
    
def k_list_time_estimator(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    # the time complexity of Wagner's algorithm should be 2^{ell + k + 1} instead of 2^{ell + k}
    # 2^{k- 1} * 2^{ell + 1} hases
    # (2^{k}- 1) * 2^{ell + 1} time for merge
    return (2^k + 2^(k-1) -  1) * 2^(1 + ell)

def k_list_mem_estimator(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    N = 2^(ell + 1)
    return ((k**2 + 5*k + 2)/4 + 2**(k-1)) * ell * N

def k_list_mem_estimator_sum(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    N = 2^(ell + 1)
    # M = \frac{N}{2} \cdot \ell \left[ (k+2) + \frac{k^2 + 3k}{2} + (2^k - 1) 
    M = ( n + ell ) * N / 2
    for i in range(k):
        M = M + (n + (2**i - i) * ell) * N / 2
    return M
    
def k_list_mem_estimator_star(n, k):
    """
    k_list algorithm with index pointer
    """
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    N = 2^(ell + 1)
    return ((k**2 + k - 2)/4 + 2**k) * ell * N

def single_list_time_estimator(n, k):
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return k * N

def single_list_mem_estimator_star(n, k):
    """
    single_list algorithm with index pointer
    """
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return 2*(n + k - ell - 1) * N

def single_list_mem_estimator_plain(n, k):
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return (2**(k-1) *(ell + 1)+ 2*ell) * N

def single_list_mem_estimator_equihash(n, k):
    # optimized version mentioned in Equihash, stores only 8 bits index in the last round.
    # the trade-off should be reconsidered
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    # (2**(k-1) *(ell + 1)+ 2*ell) * N
    return 2**(ell + 3) * (2**k + ell/2)
    
def single_list_k_upper_bound(n):
    return sqrt(n/2 + 1)

In [2]:
Equihash_Parameter_Set = [
    (96, 5),
    (128, 7),
    (160, 9),
    (96, 3),
    (144, 5),
    (150, 5),
    (200, 9),
    (288, 8)
]

def to_log2_complexity(complexity):
    """
    Convert the complexity to log2 scale.
    """
    return RR(log(complexity, 2))

def to_MB(mem_complexity):
    """
    Convert the memory complexity (bits) to MB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024)

def to_GB(mem_complexity):
    """
    Convert the memory complexity (bits) to GB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024 * 1024)

def to_TB(mem_complexity):
    """
    Convert the memory complexity (bits) to TB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024 * 1024 * 1024)

def k_list_reducded_size(n, k, t, using_trade_off=False):
    """ K-list algorithm with list size reduced to N / 2^t.
    """
    ell = n/(k + 1)
    # size of the list in height h
    if using_trade_off: 
        idx_len = 1
    else: 
        idx_len = ell - t
    N_h = lambda h: 2 ** (ell - t * 2**h)
    M = (ell + n) * N_h(0)
    T = 0
    for i in range(0, k):
        M += ((2**i) * idx_len  + n - i * ell) * N_h(i)
        T += 2**(k - 1 - i) * 2 * N_h(i)
    return M, T

def best_k_list_memory_trade_off(n, k):
    """
    The index trimming technique can be applied in K-list algorithm while not increasing the time complexity significantly. 
    We search the best index trimming bit length for K-list algorithm. We only consider the depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    M_xor = ((k**2 + 5*k + 2)/4) * ell * N
    M_idx_t = lambda t: 2**(k-1) * t * N
    M_first_run = lambda t: M_xor + M_idx_t(t)
    # N_t = lambda t: N / 2**t
    # M_second_run = lambda t: ((k**2 + 5*k + 2)/4) * ell * N_t(t) +  2**(k-1) * ell * N_t(t)
    for t in range(1, ceil(ell)):
        first_run_mem = M_first_run(t)
        second_run_mem, _ = k_list_reducded_size(n, k, t)
        if first_run_mem >= second_run_mem:
            return t, first_run_mem


In [3]:
for n, k in Equihash_Parameter_Set:
    t, op_M = best_k_list_memory_trade_off(n, k)
    plain_memory = to_log2_complexity(k_list_mem_estimator(n, k))
    equihash_memory_tf = to_log2_complexity(single_list_mem_estimator_equihash(n, k))
    equihash_memory_ip = to_log2_complexity(single_list_mem_estimator_star(n, k))
    optimized_memory = to_log2_complexity(op_M)
    print(f"k-list n = {n}, k = {k}, t = {t} optimized_memory = 2^{optimized_memory}, plain_memory = 2^{plain_memory}")
    print(f"single-list n = {n}, k = {k}, index pointer = 2^{equihash_memory_ip}, trade-off = 2^{equihash_memory_tf}")
    

k-list n = 96, k = 5, t = 1 optimized_memory = 2^24.8073549220576, plain_memory = 2^25.8579809951276
single-list n = 96, k = 5, index pointer = 2^24.3923174227788, trade-off = 2^24.3219280948874
k-list n = 128, k = 7, t = 1 optimized_memory = 2^25.6724253419715, plain_memory = 2^27.4178525148859
single-list n = 128, k = 7, index pointer = 2^24.8826430493618, trade-off = 2^26.0874628412503
k-list n = 160, k = 9, t = 1 optimized_memory = 2^26.5849625007212, plain_memory = 2^29.1699250014423
single-list n = 160, k = 9, index pointer = 2^25.2479275134436, trade-off = 2^28.0223678130285
k-list n = 96, k = 3, t = 1 optimized_memory = 2^32.3219280948874, plain_memory = 2^32.9772799234999
single-list n = 96, k = 3, index pointer = 2^32.2094533656289, trade-off = 2^31.3219280948874
k-list n = 144, k = 5, t = 1 optimized_memory = 2^33.3575520046181, plain_memory = 2^34.4429434958487
single-list n = 144, k = 5, index pointer = 2^32.9541963103869, trade-off = 2^32.4594316186373
k-list n = 150, k =

In [4]:
def best_single_list_memory_trade_off_depth2(n, k):
    """
    The index trimming technique can be applied in single-list algorithm while not increasing the time complexity significantly. 
    We will limit the XOR-removal technique below height floor(log(k,2)) such that the time penalty is within twofold.
    We search the best index trimming bit length for single-list algorithm with depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    T0 = k * N
    xr_height = floor(log(k,2))
    def M_t(t):
        if 2**(k-1) * t  + 2 * ell > n:
            return (2**(k-1) * t  + 2 * ell) * N, 0
        xor_removal_mems = [max(2**height * (ell + 1), n - (height + 1) * ell + 2**(height + 1) * t) for height in range(1, xr_height + 1)]
        xor_removal_best_mem_t = min(xor_removal_mems)
        switching_height = xor_removal_mems.index(xor_removal_best_mem_t) + 1
        M = max(2**(k-1) * t  + 2 * ell, xor_removal_best_mem_t) * N
        return M, switching_height

    best_m = +infinity
    for t in range(1, ceil(ell)):
        first_run_mem, switching_height =  M_t(t)
        second_run_mem = single_list_with_constraint(n, k, t, use_xr=1)
        mem = max(first_run_mem, second_run_mem)
        if mem < best_m:
            best_m = mem
            best_case_is_first_run = first_run_mem > second_run_mem
            best_switching_height = switching_height
            best_t = t
    print(f"{(n,k) = }, best trade-off for depth-2 with limited XOR-removal: {best_t = }, {best_switching_height = }")
    return best_t, best_m

def best_single_list_memory_trade_off_depth2_no_xor_removal(n, k):
    """
    The index trimming technique can be applied in single-list algorithm while not increasing the time complexity significantly. 
    We will not use the XOR-removal technique.
    We search the best index trimming bit length for single-list algorithm with depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    xr_height = floor(log(k,2))
    M_t = lambda t: max(2**(k-1) * t  + 2 * ell, n) * N
    best_m = +infinity
    for t in range(1, ceil(ell)):
        first_run_mem = M_t(t)
        switching_height = 0 # we do not consider XOR-removal here. 
        second_run_mem = single_list_with_constraint(n, k, t, use_xr=0)
        mem = max(first_run_mem, second_run_mem)
        if mem < best_m:
            best_m = mem
            best_case_is_first_run = first_run_mem > second_run_mem
            best_switching_height = switching_height
            best_t = t
    print(f"{(n,k) = }, best trade-off for depth-2 with no XOR-removal: {best_t = }, {best_switching_height = }")
    return best_t, best_m

def best_single_list_memory_trade_off_depth2_unlimited_xor_removal(n, k):
    """
    The index trimming technique can be applied in single-list algorithm while not increasing the time complexity significantly.
    In this case: we will apply the XOR-removal in any any layer if needed.
    We search the best index trimming bit length for single-list algorithm with the depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    def M_t(t):
        if 2**(k-1) * t  + 2 * ell > n:
            return (2**(k-1) * t  + 2 * ell) * N, 0
        xor_removal_mems = [max(2**height * (ell + 1), n - (height + 1) * ell + 2**(height + 1) * t) for height in range(1, k)]
        xor_removal_best_mem_t = min(xor_removal_mems)
        switching_height = xor_removal_mems.index(xor_removal_best_mem_t) + 1
        M = max(2**(k-1) * t  + 2 * ell, xor_removal_best_mem_t) * N
        return M, switching_height

    best_m = +infinity
    for t in range(1, ceil(ell)):
        first_run_mem, switching_height =  M_t(t)
        second_run_mem = single_list_with_constraint(n, k, t, use_xr=-1)
        mem = max(first_run_mem, second_run_mem)
        if mem < best_m:
            best_m = mem
            best_case_is_first_run = first_run_mem > second_run_mem
            best_switching_height = switching_height
            best_t = t
        # if second_run_mem < first_run_mem:
        #     return t, first_run_mem
    print(f"{(n,k) = }, best trade-off for depth-2 with unlimited XOR-removal: {best_t = }, {best_switching_height = }")
    return best_t, best_m

def single_list_with_constraint(n, k, t, use_xr = 1):
    """
    estimate the peak memory of single-list algorithm with partial t-bit solution vector constraint.
    use_xr = -1, 0, 1 ==> unlimited xor-removal, no xor-removal, limited xor-removal
    """
    # max_candi_length = lambda h: 2**(k - h)
    max_permutations = [2**t]
    max_candidates = [2**k]
    t_h = None
    ell = n/(k + 1)
    N = 2^(ell + 1)
    find = False
    for i in range(1, k):
        max_cand_i = 2**(k - i)
        max_perm_i = (2**t) ** (2**i)
        max_candidates.append(max_cand_i)
        max_permutations.append(max_perm_i)
        if max_cand_i < max_perm_i and not find:
            # the constraint will be activated at height i
            t_h = i - 1 # the list at t_h is of size O(N) and the subsequent list size will be << N.
            find = True
            # print(f"Trade-off for single-list algorithm: {n = } {k = } {t = } {t_h = }")
            # break
    def xor_removal_use_case(h):
        M1 = 2**h * (ell + 1) * N
        if use_xr == 1:
            # use xor removal in limited case
            can_xor_removal = 2**(t_h) <= k
            if not can_xor_removal:
                M1 += (n - h * ell) * N
        elif use_xr == 0:
            # do not use xor removal
             M1 += (n - h * ell) * N
        elif use_xr == -1:
            # always use xor removal
            return M1
    # memory complexity before the list size significantly reduced        
    M1 = xor_removal_use_case(t_h)
    # the reduced_size at height t_h + 1
    reduced_size_N = (max_candidates[t_h + 1] / max_permutations[t_h + 1]) * N
    M2 =  2**(t_h + 1) * (ell + 1) * reduced_size_N +  (n - (t_h + 1) * ell) * reduced_size_N
    M = max(M2, M1)
    # try next height if the reduced_size_N is not too small
    if t_h + 2 >= k - 1:
        return M
    t_h += 1
    M1 = xor_removal_use_case(t_h)
    reduced_size_N = (max_candidates[t_h + 1] / max_permutations[t_h + 1]) * binomial(reduced_size_N, 2) / 2**ell  
    M2 =  2**(t_h + 1) * (ell + 1) * reduced_size_N +  (n - (t_h + 1) * ell) * reduced_size_N
    M_prime = max(M2, M1)
    return min(M, M_prime)

for n, k in Equihash_Parameter_Set:
    t, op_M = best_single_list_memory_trade_off_depth2(n, k)
    equihash_memory_tf = to_log2_complexity(single_list_mem_estimator_equihash(n, k))
    equihash_memory_ip = to_log2_complexity(single_list_mem_estimator_star(n, k))
    optimized_memory = to_log2_complexity(op_M)
    print(f"single-list n = {n}, k = {k}, t = {t}, index pointer = 2^{equihash_memory_ip}, equihash-trade-off = 2^{equihash_memory_tf} optimized_memory = 2^{optimized_memory}")
    

(n,k) = (96, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
single-list n = 96, k = 5, t = 1, index pointer = 2^24.3923174227788, equihash-trade-off = 2^24.3219280948874 optimized_memory = 2^23.0874628412503
(n,k) = (128, 7), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 2
single-list n = 128, k = 7, t = 1, index pointer = 2^24.8826430493618, equihash-trade-off = 2^26.0874628412503 optimized_memory = 2^23.5849625007212
(n,k) = (160, 9), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 0
single-list n = 160, k = 9, t = 1, index pointer = 2^25.2479275134436, equihash-trade-off = 2^28.0223678130285 optimized_memory = 2^25.1699250014423
(n,k) = (96, 3), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
single-list n = 96, k = 3, t = 1, index pointer = 2^32.2094533656289, equihash-trade-off = 2^31.3219280948874 optimized_

In [5]:
from prettytable import PrettyTable
headers = ["(n,k)", "Single-list", "Single-list*", "Single-list-Op", "K-list", "K-list*", "K-list-Op"]
pt = PrettyTable(headers)
pt.title = "Memory Complexity for Concrete Parameters GBP(n,2^k)"
gaplen = 6
print("The paper uses the limited xor-removal case")
for n, k in Equihash_Parameter_Set:
    t_s, k_list_op_M = best_single_list_memory_trade_off_depth2(n, k)
    single_list_op_memory = to_log2_complexity(k_list_op_M)
    equihash_memory_tf = to_log2_complexity(single_list_mem_estimator_equihash(n, k))
    equihas_memory_ip = to_log2_complexity(single_list_mem_estimator_star(n, k))
    t_k, k_list_op_M = best_k_list_memory_trade_off(n, k)
    k_list_op_M = to_log2_complexity(k_list_op_M)
    k_list_memory_ip = to_log2_complexity(k_list_mem_estimator_star(n,k))
    k_list_plain = to_log2_complexity(k_list_mem_estimator(n,k))
    single_list_plain = to_log2_complexity(single_list_mem_estimator_plain(n,k))
    pt.add_row([str((n,k)),f"2^%.1f"%RR(single_list_plain), f"2^%.1f"%RR(equihas_memory_ip), f"2^%.1f"%RR(single_list_op_memory), f"2^%.1f"%RR(k_list_plain),
                f"2^%.1f"%RR(k_list_memory_ip), f"2^%.1f"%RR(k_list_op_M)])
print(pt)

The paper uses the limited xor-removal case
(n,k) = (96, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (128, 7), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 2
(n,k) = (160, 9), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (96, 3), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (144, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (150, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (200, 9), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (288, 8), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 2
+-------------------------------------------------------------------------------------

In [6]:
print(pt.get_latex_string())

\begin{tabular}{ccccccc}
(n,k) & Single-list & Single-list* & Single-list-Op & K-list & K-list* & K-list-Op \\
(96, 5) & 2^25.2 & 2^24.4 & 2^23.1 & 2^25.9 & 2^26.3 & 2^24.8 \\
(128, 7) & 2^27.1 & 2^24.9 & 2^23.6 & 2^27.4 & 2^28.1 & 2^25.7 \\
(160, 9) & 2^29.1 & 2^25.2 & 2^25.2 & 2^29.2 & 2^30.1 & 2^26.6 \\
(96, 3) & 2^32.2 & 2^32.2 & 2^30.7 & 2^33.0 & 2^33.0 & 2^32.3 \\
(144, 5) & 2^33.8 & 2^33.0 & 2^31.6 & 2^34.4 & 2^34.9 & 2^33.4 \\
(150, 5) & 2^34.9 & 2^34.0 & 2^32.7 & 2^35.5 & 2^35.9 & 2^34.4 \\
(200, 9) & 2^33.4 & 2^29.6 & 2^29.2 & 2^33.5 & 2^34.4 & 2^30.8 \\
(288, 8) & 2^45.1 & 2^42.0 & 2^40.6 & 2^45.3 & 2^46.1 & 2^42.9 \\
\end{tabular}


In [7]:
from prettytable import PrettyTable
print("No XOR Removal Case")
headers = ["(n,k)", "Single-list", "Single-list*", "Single-list-Op", "K-list", "K-list*", "K-list-Op"]
pt = PrettyTable(headers)
pt.title = "Memory Complexity for Concrete Parameters GBP(n,2^k)"
gaplen = 6
for n, k in Equihash_Parameter_Set:
    t_s, k_list_op_M = best_single_list_memory_trade_off_depth2_no_xor_removal(n, k)
    single_list_op_memory = to_log2_complexity(k_list_op_M)
    equihash_memory_tf = to_log2_complexity(single_list_mem_estimator_equihash(n, k))
    equihas_memory_ip = to_log2_complexity(single_list_mem_estimator_star(n, k))
    t_k, k_list_op_M = best_k_list_memory_trade_off(n, k)
    k_list_op_M = to_log2_complexity(k_list_op_M)
    k_list_memory_ip = to_log2_complexity(k_list_mem_estimator_star(n,k))
    k_list_plain = to_log2_complexity(k_list_mem_estimator(n,k))
    single_list_plain = to_log2_complexity(single_list_mem_estimator_plain(n,k))
    pt.add_row([str((n,k)),f"2^%.1f"%RR(single_list_plain), f"2^%.1f"%RR(equihas_memory_ip), f"2^%.1f"%RR(single_list_op_memory), f"2^%.1f"%RR(k_list_plain),
                f"2^%.1f"%RR(k_list_memory_ip), f"2^%.1f"%RR(k_list_op_M)])
print(pt)

No XOR Removal Case
(n,k) = (96, 5), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (128, 7), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (160, 9), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (96, 3), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (144, 5), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (150, 5), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (200, 9), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (288, 8), best trade-off for depth-2 with no XOR-removal: best_t = 1, best_switching_height = 0
+---------------------------------------------------------------------------------------+
|                  Memory Complexity for Concrete Parameters